# 创建用户持仓信息表并上传示例数据

本笔记本用于：
1. 创建 DynamoDB 表来存储用户持仓信息
2. 上传示例的用户持仓数据
3. 验证数据是否正确上传

## 导入必要的库

In [1]:
import boto3
import yaml
import os
import json
from datetime import datetime
import pandas as pd
from decimal import Decimal
import time

## 设置 AWS 连接

In [2]:
# 创建 DynamoDB 客户端和资源
session = boto3.Session()
region = session.region_name
dynamodb_client = boto3.client("dynamodb", region_name=region)
dynamodb_resource = boto3.resource("dynamodb", region_name=region)
smm_client = boto3.client("ssm")

print(f"AWS Region: {region}")

AWS Region: us-east-1


## 定义知识库名称和表名

In [3]:
kb_name = "fsi-fund-knowledge"
table_name = "user_holdings"
pk_item = "user_id"
sk_item = "fund_code"

## 创建 DynamoDB 表

In [4]:
def create_dynamodb_table(kb_name, table_name, pk_item, sk_item):
    """创建 DynamoDB 表
    Args:
        kb_name: 知识库名称，用于创建 SSM 参数
        table_name: 表名
        pk_item: 主键名称
        sk_item: 排序键名称
    """
    try:
        table = dynamodb_resource.create_table(
            TableName=table_name,
            KeySchema=[
                {"AttributeName": pk_item, "KeyType": "HASH"},
                {"AttributeName": sk_item, "KeyType": "RANGE"},
            ],
            AttributeDefinitions=[
                {"AttributeName": pk_item, "AttributeType": "S"},
                {"AttributeName": sk_item, "AttributeType": "S"},
            ],
            BillingMode="PAY_PER_REQUEST",  # 使用按需容量模式
        )

        # 等待表创建完成
        print(f"正在创建表 {table_name}...")
        table.wait_until_exists()
        print(f"表 {table_name} 创建成功！")
        
        # 在参数存储中保存表名
        smm_client.put_parameter(
            Name=f"{kb_name}-{table_name}-table-name",
            Description=f"{kb_name} {table_name} 表名",
            Value=table_name,
            Type="String",
            Overwrite=True,
        )
        print(f"参数 {kb_name}-{table_name}-table-name 创建成功！")
        return table
    except dynamodb_client.exceptions.ResourceInUseException:
        print(f"表 {table_name} 已存在，跳过表创建步骤")
        # 在参数存储中保存表名
        smm_client.put_parameter(
            Name=f"{kb_name}-{table_name}-table-name",
            Description=f"{kb_name} {table_name} 表名",
            Value=table_name,
            Type="String",
            Overwrite=True,
        )
        print(f"参数 {kb_name}-{table_name}-table-name 创建成功！")
        return dynamodb_resource.Table(table_name)

In [5]:
# 创建用户持仓表
user_holdings_table = create_dynamodb_table(kb_name, table_name, pk_item, sk_item)

正在创建表 user_holdings...
表 user_holdings 创建成功！
参数 fsi-fund-knowledge-user_holdings-table-name 创建成功！


## 准备示例用户持仓数据

In [6]:
# 创建示例用户持仓数据
sample_holdings = [
    {
        "user_id": "user001",
        "fund_code": "000001",
        "fund_name": "华夏成长混合",
        "holding_amount": Decimal("1000.00"),
        "purchase_date": "2025-01-15",
        "purchase_price": Decimal("1.5432"),
        "current_value": Decimal("1600.50"),
        "profit_loss": Decimal("57.00"),
        "last_updated": datetime.now().isoformat()
    },
    {
        "user_id": "user001",
        "fund_code": "000002",
        "fund_name": "易方达策略成长混合",
        "holding_amount": Decimal("2000.00"),
        "purchase_date": "2025-02-20",
        "purchase_price": Decimal("2.1234"),
        "current_value": Decimal("2300.75"),
        "profit_loss": Decimal("76.50"),
        "last_updated": datetime.now().isoformat()
    },
    {
        "user_id": "user001",
        "fund_code": "000003",
        "fund_name": "南方优选成长混合",
        "holding_amount": Decimal("1500.00"),
        "purchase_date": "2025-03-10",
        "purchase_price": Decimal("1.8765"),
        "current_value": Decimal("1750.25"),
        "profit_loss": Decimal("43.75"),
        "last_updated": datetime.now().isoformat()
    },
    {
        "user_id": "user002",
        "fund_code": "000001",
        "fund_name": "华夏成长混合",
        "holding_amount": Decimal("3000.00"),
        "purchase_date": "2025-01-05",
        "purchase_price": Decimal("1.5000"),
        "current_value": Decimal("4800.00"),
        "profit_loss": Decimal("300.00"),
        "last_updated": datetime.now().isoformat()
    },
    {
        "user_id": "user002",
        "fund_code": "000004",
        "fund_name": "博时主题行业混合",
        "holding_amount": Decimal("2500.00"),
        "purchase_date": "2025-04-01",
        "purchase_price": Decimal("2.5678"),
        "current_value": Decimal("2800.50"),
        "profit_loss": Decimal("120.25"),
        "last_updated": datetime.now().isoformat()
    },
    {
        "user_id": "user003",
        "fund_code": "000005",
        "fund_name": "嘉实增长混合",
        "holding_amount": Decimal("5000.00"),
        "purchase_date": "2025-05-01",
        "purchase_price": Decimal("1.2345"),
        "current_value": Decimal("5500.00"),
        "profit_loss": Decimal("172.50"),
        "last_updated": datetime.now().isoformat()
    }
]

## 上传示例数据到 DynamoDB 表

In [7]:
def upload_items_to_dynamodb(table, items):
    """上传多个项目到 DynamoDB 表
    Args:
        table: DynamoDB 表资源
        items: 要上传的项目列表
    """
    for item in items:
        try:
            table.put_item(Item=item)
            print(f"成功上传项目: {item['user_id']} - {item['fund_code']}")
        except Exception as e:
            print(f"上传项目时出错: {str(e)}")
            print(f"项目: {item}")

In [8]:
# 上传示例数据
upload_items_to_dynamodb(user_holdings_table, sample_holdings)

成功上传项目: user001 - 000001
成功上传项目: user001 - 000002
成功上传项目: user001 - 000003
成功上传项目: user002 - 000001
成功上传项目: user002 - 000004
成功上传项目: user003 - 000005


## 验证数据是否正确上传

In [9]:
def query_user_holdings(table, user_id):
    """查询用户持仓信息
    Args:
        table: DynamoDB 表资源
        user_id: 用户ID
    Returns:
        用户持仓信息列表
    """
    response = table.query(
        KeyConditionExpression=boto3.dynamodb.conditions.Key("user_id").eq(user_id)
    )
    return response["Items"]

In [10]:
# 查询用户001的持仓信息
user001_holdings = query_user_holdings(user_holdings_table, "user001")
print("用户001的持仓信息:")
for holding in user001_holdings:
    print(f"基金代码: {holding['fund_code']}, 基金名称: {holding['fund_name']}, 持有份额: {holding['holding_amount']}, 当前价值: {holding['current_value']}")

用户001的持仓信息:
基金代码: 000001, 基金名称: 华夏成长混合, 持有份额: 1000, 当前价值: 1600.5
基金代码: 000002, 基金名称: 易方达策略成长混合, 持有份额: 2000, 当前价值: 2300.75
基金代码: 000003, 基金名称: 南方优选成长混合, 持有份额: 1500, 当前价值: 1750.25


In [11]:
# 查询用户002的持仓信息
user002_holdings = query_user_holdings(user_holdings_table, "user002")
print("用户002的持仓信息:")
for holding in user002_holdings:
    print(f"基金代码: {holding['fund_code']}, 基金名称: {holding['fund_name']}, 持有份额: {holding['holding_amount']}, 当前价值: {holding['current_value']}")

用户002的持仓信息:
基金代码: 000001, 基金名称: 华夏成长混合, 持有份额: 3000, 当前价值: 4800
基金代码: 000004, 基金名称: 博时主题行业混合, 持有份额: 2500, 当前价值: 2800.5


In [12]:
# 查询用户003的持仓信息
user003_holdings = query_user_holdings(user_holdings_table, "user003")
print("用户003的持仓信息:")
for holding in user003_holdings:
    print(f"基金代码: {holding['fund_code']}, 基金名称: {holding['fund_name']}, 持有份额: {holding['holding_amount']}, 当前价值: {holding['current_value']}")

用户003的持仓信息:
基金代码: 000005, 基金名称: 嘉实增长混合, 持有份额: 5000, 当前价值: 5500


## 更新 prereqs_config.yaml 文件

请在 `fund-advisor-agent-strands/prereqs/prereqs_config.yaml` 文件中添加以下内容：

```yaml
  # 用户持仓信息表
  - table_name: "user_holdings"
    pk_item: "user_id"
    sk_item: "fund_code"
```

## 使用 user_info.py 中的工具函数测试

In [ ]:
import sys
import os

# 添加项目根目录到 Python 路径
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

# 导入 user_info 模块中的函数
from tools.user_info import get_user_holdings, get_user_fund_holding, get_user_portfolio_summary

In [ ]:
# 测试获取用户持仓信息
user001_holdings = get_user_holdings("user001")
print("用户001的持仓信息:")
print(json.dumps(user001_holdings, indent=2, default=str))

In [ ]:
# 测试获取用户特定基金的持仓信息
user001_fund_holding = get_user_fund_holding("user001", "000001")
print("用户001持有的基金000001信息:")
print(json.dumps(user001_fund_holding, indent=2, default=str))

In [ ]:
# 测试获取用户投资组合摘要
user001_portfolio = get_user_portfolio_summary("user001")
print("用户001的投资组合摘要:")
print(json.dumps(user001_portfolio, indent=2, default=str))